# **Welcome to the ABDICO clustering Notebook**

* This notebook helps recognize dominant groups of related actors, objects, regulated actions ("aims") and modals ("deontics") in institutional statements.
Compared to traditional word based topic modeling such as LDA, we use BERTopic, which pursues a semantic ('word meaning') based approach.

### **This notebook performs the following tasks**


* It takes a main.csv file which has columns designated as "Attribute", "Objects", "Deontics" and "Aims" respectively
* User indicates the institutional constituent (ABDICO) over which clustering is to be performed
* Output main.csv contains a "_group" column, indicating the respective topic cluster to which the constituent belongs.
* The categorical topic of the group is indicated by the top most representative words from the cluster.








# **Installations and Setup**
* This code sets up the analysis. You don't have to understand it. Just run it and then scroll down.
* These commands below install the necessary components for the rest of the analysis to work. To run press ***ctrl+enter*** keys or select ***Runtime*** from the menu above and then one of the ***Run*** options within it.

In [8]:
!git clone https://github.com/BSAkash/NLP4GOV
!pip install transformers
!pip install sentence-transformers
!pip install bertopic

Cloning into 'NLP4GOV'...
remote: Enumerating objects: 2154, done.
remote: Counting objects: 100% (310/310), done.
remote: Compressing objects: 100% (167/167), done.
remote: Total 2154 (delta 164), reused 285 (delta 141), pack-reused 1844
Receiving objects: 100% (2154/2154), 7.98 MiB | 24.11 MiB/s, done.
Resolving deltas: 100% (1263/1263), done.


# **Data upload**

Run this cell to run your own search engine.

For your own data, you will likely have to adapt it for this notebook to run. See below for the sample format

Please name uploaded files as `main.csv` and `main2.csv`. For each institutional statement, there must be columns indicating the corresponding ABDICO codings. That is "Attribute", "Object", "Deontic", "Aim". These may be hancoded or computationally extracted.

See ABDICO_parsing.ipynb in this repository for automated institutional grammar coding of policies.


In [9]:
from google.colab import files
uploaded = files.upload()

# **Or use our data archives**

You may also uncomment and run the cell below to follow the demonstration on archival data. This shall directly download (into Colab) our datasets for the Apache Software Foundation, an open source software community.

* [ABDICO coded Apache community policies](https://storage.googleapis.com/cscw_2022/anamika_os.csv)

In [10]:
##Else you can directly use the !wget command below to download our datasets into the code notebook,
##make sure you uncomment the below code by (ctrl+/) keys before running this cell of code

# # ABDICO coded policy example
# !wget -O main.csv https://storage.googleapis.com/public_data_c2/IG_datasets/ASF_ABDICO.csv
# import pandas as pd
# import nltk
# nltk.download('punkt')
# data = pd.read_csv('main.csv')
# data['raw institutional statement'] = data['raw institutional statement'].apply(lambda x : nltk.tokenize.sent_tokenize(x))
# data = data.explode('raw institutional statement')
# data.to_csv('main.csv',index=False)

--2024-03-21 08:29:02--  https://storage.googleapis.com/public_data_c2/IG_datasets/ASF_ABDICO.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.107.207, 74.125.20.207, 108.177.98.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.107.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 359735 (351K) [text/csv]
Saving to: ‘main.csv’

main.csv            100%[===================>] 351.30K  --.-KB/s    in 0.1s    

2024-03-21 08:29:02 (3.22 MB/s) - ‘main.csv’ saved [359735/359735]



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# **Clustering and topic modeling**

* Prior to uploading the your own dataset remove all the NaN('nan' in python represents missing or undefined data value typically something that is not a number) text value rows from the csv file.
* You may need to make some changes to the first three lines of the following cell:

  * **Component** : The insitutional component of policies/institutional statements you would like to cluster and group. This should exactly match the column in which data which contains the ABDICO component of interest. Case Sensitive.
  * **top_n_words** : The N dominant words to indidate/represent the topic of the group. We recommend setting this number no more than 10.
  * **num_topic** : Now this one is a little tricky. This value guides the clustering alogithm to find the specified number of clusters and topics among the ABDICO components. You may use an estimate (e.g. if you are clustering attributes and are aware that there are around 5 different actors in your policy data), or set a slightly higher number. It's simpler to manually combine some similar clusters. Our default is 20, for our apache policies.

In [11]:
Component = "Object"
top_n_words = 3
num_topic = 20

# clustering of components
from bertopic import BERTopic
import pandas as pd
result = pd.read_csv('main.csv', usecols=[Component])
result.dropna(subset=[Component],inplace=True)
components = result[Component].tolist()

topic_model = BERTopic(top_n_words = 5,nr_topics = num_topic)
topic_model.hdbscan_model.gen_min_span_tree=True
topic_model.umap_model.random_state= 0 ##set seed to enable reproduction of clustering

topic_model.fit(components)
freq = topic_model.get_topic_info()
result[Component + '_group'] = topic_model.transform(components)[0]
result[Component + '_group'] = result[Component + '_group'].apply(lambda x : freq.loc[x+1,'Name'])
result.to_csv('main.csv', index=False)

In [12]:
files.download('main.csv')#download file here

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
#Generate Apache Community Network. Minor differences in topics may be 
#due to UMAP stochasticitiy: https://umap-learn.readthedocs.io/en/latest/reproducibility.html
%cd /content
%run -i  ./NLP4GOV/ABDICO_clustering/gen_ASF_graph.py

### Going deeper
Check out [BERTopic](https://maartengr.github.io/BERTopic/) documentation for more arguments, parameters and methods for sophisticated topic modeling. For "fine tuning" your topic modeling, see our [work](https://arxiv.org/abs/2309.14245) on governance of open source software.